In [5]:
import pysam
import gffutils

In [14]:
gtf_file = 'genes.gtf'
bam_files = [
    'shorterCV1_2.bam',
] #318,553 mapped reads
output_file_path = 'count_table_v2.txt'

#3247

bedtools intersect -a tss_regions_mm10.bed -b shorterCV1.sort.bam -bed -wa -wb > intersectedCV1.bed

awk '{print $4}' intersectedCV1.bed | sort | uniq -c > read_counts_per_gene.txt

In [10]:
def check_bam_headers(bam_files):
    for bam_file in bam_files:
        samfile = pysam.AlignmentFile(bam_file, "rb")
        print(f"Header for {bam_file}:")
        for ref in samfile.header.references:
            print(f"{ref} with length {samfile.header.get_reference_length(ref)}")
        samfile.close()


# Check BAM file headers to understand the chromosome naming and sizes
check_bam_headers(bam_files)

Header for shorterCV1.bam:
chr1 with length 195471971
chr2 with length 182113224
chr3 with length 160039680
chr4 with length 156508116
chr5 with length 151834684
chr6 with length 149736546
chr7 with length 145441459
chr8 with length 129401213
chr9 with length 124595110
chr10 with length 130694993
chr11 with length 122082543
chr12 with length 120129022
chr13 with length 120421639
chr14 with length 124902244
chr15 with length 104043685
chr16 with length 98207768
chr17 with length 94987271
chr18 with length 90702639
chr19 with length 61431566
chrX with length 171031299
chrY with length 91744698
chrM with length 16299
chr1_GL456210_random with length 169725
chr1_GL456211_random with length 241735
chr1_GL456212_random with length 153618
chr1_GL456213_random with length 39340
chr1_GL456221_random with length 206961
chr4_GL456216_random with length 66673
chr4_GL456350_random with length 227966
chr4_JH584292_random with length 14945
chr4_JH584293_random with length 207968
chr4_JH584294_random 

In [15]:
def get_tss_positions(gtf_file):
    db = gffutils.create_db(gtf_file, dbfn=':memory:', force=True, keep_order=True, merge_strategy='merge', sort_attribute_values=True)
    tss_positions = {}
    for gene in db.features_of_type('gene'):
        exons = list(db.children(gene, featuretype='exon', order_by='start'))
        if not exons:
            continue
        chromosome = gene.seqid
        tss = exons[0].start if gene.strand == '+' else exons[-1].end
        gene_id = gene.attributes['gene_id'][0]
        tss_positions[(chromosome, gene_id)] = tss
    return tss_positions

tss_positions = get_tss_positions(gtf_file)


In [13]:
lengd = len(tss_positions)

lengd

23337

In [37]:
def count_reads_around_tss(bam_file, tss_positions):
    samfile = pysam.AlignmentFile(bam_file, "rb")
    counts = {}
    for (chrom, gene_id), tss in tss_positions.items():
        key = (chrom, gene_id)
        start = max(0, tss - 2000)
        end = tss + 2000
        counts[key] = 0
        print(f"Processing {chrom}:{start}-{end} for {gene_id} in {bam_file}")
        try:
            for pileupcolumn in samfile.pileup(region=f"{chrom}:{start}-{end}"):
                counts[key] += pileupcolumn.n
        except ValueError as e:
            print(f"Error processing region {chrom}:{start}-{end} for {gene_id} in {bam_file}: {e}")
    samfile.close()
    return counts

samfile = pysam.AlignmentFile(bam_files[0], "rb")
columns = samfile.pileup(region="chr6:72345317-72349317")

for something in columns:
    print(something)

5	72348168	1
SNL138:273:C8CA7ACXX:5:1106:0:171550544	16	#5	72348169	42	49M	*	0	0	TATGTCTGTTTCCCTAGCCAGAGAGGTGAATAAGGTGTCCAACATCACA	array('B', [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 37, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 33, 33, 33])	[('AS', 0), ('XN', 0), ('XM', 0), ('XO', 0), ('XG', 0), ('NM', 0), ('MD', '49'), ('YT', 'UU')]	0	0	125862400	0	1	0	0
5	72348169	1
SNL138:273:C8CA7ACXX:5:1106:0:171550544	16	#5	72348169	42	49M	*	0	0	TATGTCTGTTTCCCTAGCCAGAGAGGTGAATAAGGTGTCCAACATCACA	array('B', [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 37, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 33, 33, 33])	[('AS', 0), ('XN', 0), ('XM', 0), ('XO', 0), ('XG', 0), ('NM', 0), ('MD', '49'), ('YT', 'UU')]	1	0	125862400	0	0	0	0
5	72348170	1
SNL138:273:C8CA7ACXX:5:1106:0:171550544	16	#5	72348169	42	49M	*	0	0	TATGT

In [36]:
# for bam_file in bam_files:
#     counts = count_reads_around_tss(bam_file, tss_positions)
#     for key in counts:
#         if key in all_counts:
#             all_counts[key].append(counts[key])
#         else:
#             all_counts[key] = [counts[key]]

all_counts = {}
counts = count_reads_around_tss(bam_files[0], tss_positions)

Processing chr7:45573176-45577176 for 0610005C13Rik in shorterCV1_2.bam
Processing chr5:31046640-31050640 for 0610007C21Rik in shorterCV1_2.bam
Processing chr5:130217744-130221744 for 0610007L01Rik in shorterCV1_2.bam
Processing chr15:32242662-32246662 for 0610007N19Rik in shorterCV1_2.bam
Processing chr13:63813320-63817320 for 0610007P08Rik in shorterCV1_2.bam
Processing chr12:85822545-85826545 for 0610007P14Rik in shorterCV1_2.bam
Processing chr17:25238170-25242170 for 0610007P22Rik in shorterCV1_2.bam
Processing chr2:163539721-163543721 for 0610008F07Rik in shorterCV1_2.bam
Processing chr12:80690466-80694466 for 0610009B14Rik in shorterCV1_2.bam
Processing chr11:51686634-51690634 for 0610009B22Rik in shorterCV1_2.bam
Processing chr12:4815608-4819608 for 0610009D07Rik in shorterCV1_2.bam
Processing chr11:120346678-120350678 for 0610009L18Rik in shorterCV1_2.bam
Processing chr18:38248249-38252249 for 0610009O20Rik in shorterCV1_2.bam
Processing chr2:176634319-176638319 for 0610010B08R

In [35]:
counts

{('chr7', '0610005C13Rik'): 0,
 ('chr5', '0610007C21Rik'): 0,
 ('chr5', '0610007L01Rik'): 0,
 ('chr15', '0610007N19Rik'): 0,
 ('chr13', '0610007P08Rik'): 0,
 ('chr12', '0610007P14Rik'): 0,
 ('chr17', '0610007P22Rik'): 0,
 ('chr2', '0610008F07Rik'): 0,
 ('chr12', '0610009B14Rik'): 0,
 ('chr11', '0610009B22Rik'): 0,
 ('chr12', '0610009D07Rik'): 0,
 ('chr11', '0610009L18Rik'): 0,
 ('chr18', '0610009O20Rik'): 0,
 ('chr2', '0610010B08Rik'): 0,
 ('chr11', '0610010F05Rik'): 0,
 ('chr11', '0610010K14Rik'): 0,
 ('chr18', '0610010O12Rik'): 0,
 ('chr17', '0610011F06Rik'): 0,
 ('chr2', '0610011L14Rik'): 0,
 ('chr16', '0610012G03Rik'): 0,
 ('chr2', '0610012H03Rik'): 0,
 ('chr6', '0610030E20Rik'): 49,
 ('chr3', '0610031J06Rik'): 0,
 ('chr3', '0610031O16Rik'): 0,
 ('chr4', '0610037L13Rik'): 0,
 ('chr16', '0610037P05Rik'): 0,
 ('chr8', '0610038B21Rik'): 0,
 ('chr17', '0610038L08Rik'): 0,
 ('chr2', '0610039K10Rik'): 0,
 ('chr5', '0610040B10Rik'): 0,
 ('chr6', '0610040F04Rik'): 0,
 ('chr5', '0610040J01R

In [ ]:
all_counts = {}
for bam_file in bam_files:
    counts = count_reads_around_tss(bam_file, tss_positions)
    for key in counts:
        if key in all_counts:
            all_counts[key].append(counts[key])
        else:
            all_counts[key] = [counts[key]]

In [ ]:

# Check BAM file headers to understand the chromosome naming and sizes
check_bam_headers(bam_files)

tss_positions = get_tss_positions(gtf_file)
all_counts = {}
for bam_file in bam_files:
    counts = count_reads_around_tss(bam_file, tss_positions)
    for key in counts:
        if key in all_counts:
            all_counts[key].append(counts[key])
        else:
            all_counts[key] = [counts[key]]

with open(output_file_path, 'w') as output_file:
    header = "Gene ID\t" + "\t".join(bam_files) + "\n"
    output_file.write(header)
    for (chrom, gene_id), counts in all_counts.items():
        line = f'{gene_id}\t' + "\t".join(map(str, counts)) + '\n'
        output_file.write(line)